In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow

class LogisticRegressionModel:
    def __init__(self, C=1.0, penalty='l2'):
        self.C = C
        self.penalty = penalty
        self.model = None
        self.tracking = mlflow.set_tracking_uri("http://localhost:5000")
        self.experiment = mlflow.set_experiment("NeuralNetwork")
    
    def _log_metrics(self, y_true, y_pred):
        accuracy = accuracy_score(y_true, y_pred)
        #precision = precision_score(y_true, y_pred, average='weighted')
        #recall = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')

        mlflow.log_metrics({
            'accuracy': accuracy,
            #'precision': precision,
            #'recall': recall,
            'f1': f1
        })
        
    def _log_params(self):
        mlflow.log_params({
            'C': self.C,
            'penalty': self.penalty
        })
        
    def train(self, X_train, y_train, X_val=None, y_val=None, max_iter=100):
        with mlflow.start_run() as run:
            self.model = LogisticRegression(C=self.C, penalty=self.penalty, max_iter=max_iter)
            self.model.fit(X_train, y_train)
            
            y_pred_train = self.model.predict_proba(X_train)[:, 1]
            #self._log_metrics(y_train, y_pred_train)
            
            if X_val is not None and y_val is not None:
                y_pred_val = self.model.predict(X_val)
                self._log_metrics(y_val, y_pred_val)
                
            self._log_params()
            
            mlflow.sklearn.log_model(self.model, 'model')
            return y_pred_train
            
    def predict(self, X, threshold=0.5):
        return (self.model.predict_proba(X)[:, 1] >= threshold).astype(int)


In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load iris dataset
data = pd.read_csv('dados.csv')
X = data.drop('downgrade', axis=1)
y = data['downgrade']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
# Create instance of LogisticRegressionModel
logistic_regression = LogisticRegressionModel()

In [59]:
# Train model and log metrics and parameters
logistic_regression.train(X_train, y_train)

array([0.9999935 , 0.9102037 , 0.99873085, ..., 0.99853545, 0.81491121,
       0.73008308])

In [60]:
logistic_regression.predict(X_train)

array([1, 1, 1, ..., 1, 1, 1])

In [29]:

# Make predictions on test set
y_pred = logistic_regression.predict(X_train)

AttributeError: 'NoneType' object has no attribute 'predict'